In [1]:
import numpy as np

### Cadena de Markov

<img src="img/q1.jpeg" />


Además de usar políticas y gradientes, también podemos explicar el comportamiento de un agente como un grafo; mejor aún, como una cadena de Márkov.

El siguiente dibujo establece un escenario básico donde un agente puede pasar de un estado s0 a un estado s1 a través de una acción a2 que tiene una probabilidad de 0.2… como podemos observar las transiciones entre estados se ejecutan mediante acciones que están sujetas a una probabilidad y además, algunas transiciones tienen rewards (negativos y positivos).

El siguiente código representa las probabilidades de las transiciones como una matriz: 

In [2]:
np.random.seed(42)

transition_probabilities = [ # shape=[s, s']
        [0.7, 0.2, 0.0, 0.1],  # from s0 to s0, s1, s2, s3
        [0.0, 0.0, 0.9, 0.1],  # from s1 to ...
        [0.0, 1.0, 0.0, 0.0],  # from s2 to ...
        [0.0, 0.0, 0.0, 1.0]]  # from s3 to ...

n_max_steps = 50

def print_sequence():
    current_state = 0
    print("States:", end=" ")
    for step in range(n_max_steps):
        print(current_state, end=" ")
        if current_state == 3:
            break
        current_state = np.random.choice(range(4), p=transition_probabilities[current_state])
    else:
        print("...", end="")
    print()

for _ in range(10):
    print_sequence()

States: 0 0 3 
States: 0 1 2 1 2 1 2 1 2 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 ...
States: 0 0 3 
States: 0 0 0 1 2 1 2 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 


### MDP: Proceso de Desicion de Markov:

Definamos algunas probabilidades de transición, recompensas y posibles acciones. Por ejemplo, en el estado s0, si se elige la acción a0 entonces con proba 0.7 pasaremos al estado s0 con recompensa +10, con probabilidad 0.3 pasaremos al estado s1 sin recompensa, y nunca pasaremos al estado s2 (entonces el las probabilidades de transición son [0.7, 0.3, 0.0] y las recompensas son [+10, 0, 0]) (similar a lo que vemos en el diagrama anterior):

In [3]:
# T(s,a,s')
transition_probabilities = [ # shape=[s, a, s']
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
        [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
        [None, [0.8, 0.1, 0.1], None]]

# R(s,a,s')
rewards = [ # shape=[s, a, s']
        [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
        [[0, 0, 0], [+40, 0, 0], [0, 0, 0]]]

possible_actions = [[0, 1, 2], [0, 2], [1]]

Afortunadamente, podemos usar la función de optimalidad de Bellman (Bellman optimality Equation) para encontrar el estado mas optimo a partir de un estado  s.

- $V^*(s) = max_a \sum_s{T(s,a,s')[R(s,a,s')+\gamma \dot V^*(s')]} $ *para todo s*

donde:

- $T(s,a,s')$ es la probabilidad de transicion del estado s a s' dada una accion a.
- $R(s,a,s')$ es el reward que gana el agente de ir del stado s a s'.
- $\gamma$ es el factor de descuento

De esta formula, podemos dericar un algoritmo que permite estimar el estado optimo a partir de cada estado s. El algoritmo de Iteracion de Valor (Value Iteration Algorithm).  VAI sin embargo tiene un problema, a pesar de que puede encontrar el estado optimo s', este no necesariamente encuentra la mejor solucion en general.

- $V^*_{k+1}(s) = max_a \sum_s'{T(s,a,s')[R(s,a,s')+\gamma \dot V^*(s')]} $ *para todo s*

Aqui $V^*_{k}(s)$ es el valor estimado del estado s en la iteracion k del algoritmo.

Al notar este problema, Bellman encontro una solucion al problema al cual llamo Q-Values (Quality Values). El algoritmo es similar a VAI con algunas diferencias:

- $Q_{k+1}(s,1) = \sum_s'{T(s,a,s')[R(s,a,s')+\gamma \dot max_{a'} Q_k(s',a')]} $ *para todo s'a*

Los estados optimos calculados por Q-Values, definen la politica basica que debe usar el agente. Por tanto si un agente esta en un estado s, la accion a utilizar esta definida por:

- $\pi^*(s)= argmax_a(Q^*(s,a))$

Vamos a desarrollar un codigo basico para representar la generacion de los Q-Values.

La política óptima para este MDP, cuando se usa un factor de descuento de 0.90, es elegir la acción a0 cuando está en el estado s0, y elegir la acción a0 cuando está en el estado s1, y finalmente elegir la acción a1 (la única acción posible) cuando está en el estado s2.

In [4]:
Q_values = np.full((3, 3), -np.inf) # -np.inf for impossible actions
for state, actions in enumerate(possible_actions):
    Q_values[state, actions] = 0.0  # for all possible actions
    
print(Q_values)

[[  0.   0.   0.]
 [  0. -inf   0.]
 [-inf   0. -inf]]


In [5]:
gamma = 0.95  # the discount factor

for iteration in range(50):
    Q_prev = Q_values.copy()
    for s in range(3):
        for a in possible_actions[s]:
            Q_values[s, a] = np.sum([
                    transition_probabilities[s][a][sp]
                    * (rewards[s][a][sp] + gamma * np.max(Q_prev[sp]))
                for sp in range(3)])
            
print(Q_values)

[[21.73304188 20.63807938 16.70138772]
 [ 0.95462106        -inf  1.01361207]
 [       -inf 53.70728682        -inf]]


Vamos a utilizar Q-Values para determinar la proxima accion desde s1

In [6]:
np.argmax(Q_values, axis=1)

array([0, 2, 1], dtype=int64)

¡Ahora la política ha cambiado! En el estado s1, ahora preferimos atravesar el fuego (elija la acción a2). Esto se debe a que el factor de descuento es mayor, por lo que el agente valora más el futuro y, por lo tanto, está listo para pagar una multa inmediata para obtener más recompensas futuras.

### Q-Learning

Hasta el momento, ha sido sencillo configurar el agente porque tenemos un diagrama de las probabilidades de transición, estados y rewards, sin embargo esto no sucede en la vida real. El agente debe por si solo, construir estas estructuras mediante prueba y error.

Q-Learning funciona observando a un agente jugar (por ejemplo, de forma random) y mejorando gradualmente sus estimaciones de los Q-Values. Una vez que tiene estimaciones precisas de Q-Value (o lo suficientemente cerca), entonces la política óptima consiste en elegir la acción que tiene el Q-Value más alto .

Necesitaremos simular un agente moviéndose en el entorno, así que definamos una función para realizar alguna acción y obtener el nuevo estado y una recompensa:  

In [13]:
# esta funcion basicamente escoge el proximo estado s' de forma aleatoria 
# y devuelve el estado y el reward.
def step(state, action):
    probas = transition_probabilities[state][action]
    next_state = np.random.choice([0, 1, 2], p=probas)
    reward = rewards[state][action][next_state]
    return next_state, reward

# tambien necesitamos explorar todos los estados varas veces para obtener los estados s'.
def exploration_policy(state):
    return np.random.choice(possible_actions[state])

# Ejecucion del algoritmo. 
# vamos a crear la tabla (matriz) de los Q-values
np.random.seed(42)

Q_values = np.full((3, 3), -np.inf)
for state, actions in enumerate(possible_actions):
    Q_values[state][actions] = 0

alpha0 = 0.05 # initial learning rate
decay = 0.005 # learning rate decay
gamma = 0.90 # discount factor
state = 0 # initial state
history2 = [] # Not shown in the book

for iteration in range(10000):
    history2.append(Q_values.copy())
    action = exploration_policy(state)
    next_state, reward = step(state, action)
    next_value = np.max(Q_values[next_state]) # greedy policy at the next step
    alpha = alpha0 / (1 + iteration * decay)
    Q_values[state, action] *= 1 - alpha
    Q_values[state, action] += alpha * (reward + gamma * next_value)
    state = next_state

history2 = np.array(history2)

print(Q_values)

[[18.77621289 17.2238872  13.74543343]
 [ 0.                -inf -8.00485647]
 [       -inf 49.40208921        -inf]]


In [14]:
# accion optima para cada estado:
np.argmax(Q_values, axis=1) # optimal action for each state

array([0, 0, 1], dtype=int64)